<a href="https://colab.research.google.com/github/AbshkPskr/Zomato-Analysis/blob/master/z_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/AbshkPskr/Zomato-Analysis/master/reviews.csv",names = ['name','rating','review','sentiment']).drop_duplicates()

In [3]:
df.name.unique()

array(['Master Of Malts', 'Key - Hotel Samrat', "Mama's Buoi",
       'Lost Lemons', 'The Dineroom', "Privee'", 'The Beer Cafe',
       'Distillery - Craft Beer & Cocktails', 'Kakapo',
       'Kalaakaar - The Melodramatic Cafe',
       'Club BW - The Suryaa New Delhi',
       'The Hong Kong Club - Andaz Delhi', 'Chevron',
       'TOS - Take Off Scarlet', 'MAD Music And Drinks',
       'Burger Singh - Big Punjabi Burgers', "La Pino'z Pizza",
       'Warehouse Cafe', 'Bo Tai - Switch',
       'Raftaar - The High Speed Lounge & Bar', 'Local', 'Pehle Chai',
       'Cafe Maddox', 'Elation', 'PCO',
       "Baba's Beans Coffee Bar - Taj Vivanta", 'MIC Drop',
       'The Chatter House', 'The Sky High',
       'n the latest in...read more"'], dtype=object)

In [4]:
df = df[df.name != 'n the latest in...read more"']

In [5]:
df

,name,rating,review,sentiment
0,Master Of Malts,4.4,Worst experience ever . Zero corporation from ...,-0.433333
1,Master Of Malts,4.4,Their ajwain fish tikka is just awesome 😍🔥👌 it...,0.600000
2,Key - Hotel Samrat,3.6,Amazing experience ❤️,0.600000
3,Key - Hotel Samrat,3.6,Love the place but coz of the whole Disney the...,0.252000
4,Key - Hotel Samrat,3.6,Superb place for Parties! The best thing is th...,0.843750
...,...,...,...,...
22055,Local,4.3,"Hey team, pl add bar menu also...so that other...",0.318182
22056,Local,4.3,This is a new place which opened last Saturday...,0.089394
22057,Local,4.3,Wish there were negative ratings. The place su...,-0.325000
22058,Local,4.3,Thank god delhi has some place where one can h...,0.242105


In [6]:
df[df.name == 'Local'].rating.unique()

array([4.3])

In [7]:
bar = df.groupby('name').count().sort_values(by = 'rating')
bar

,rating,review,sentiment
name,,,
Burger Singh - Big Punjabi Burgers,1,1,1
Pehle Chai,4,4,4
Bo Tai - Switch,5,5,5
Kakapo,8,8,8
The Beer Cafe,21,21,21
Baba's Beans Coffee Bar - Taj Vivanta,22,22,22
MAD Music And Drinks,29,29,29
Key - Hotel Samrat,38,38,38
Kalaakaar - The Melodramatic Cafe,44,44,44


In [8]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=bar.index,
#                          y=bar.rating,
#                          mode='lines',
#                          name='confirmed',line=dict( width=4)))
# fig.update_layout(margin=dict(l=0,r=20,b=0,t=60,pad=0),
#                   paper_bgcolor="white",height= 600,
#                   legend=dict(x=.01,y=.98),
#                   title_text = 'reviews',font_size=15,
#                   xaxis_title="name",
#                   yaxis_title="Number of reviews")

In [9]:
!pip install jupyter-dash -q
!pip install dash-dangerously-set-inner-html -q

In [10]:
# import plotly.express as px
# from jupyter_dash import JupyterDash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# import dash
# from  dash_dangerously_set_inner_html import DangerouslySetInnerHTML as DSIH

# # Load Data
# # df = px.data.tips()
# # Build App
# external_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = JupyterDash(__name__,external_stylesheets=external_css)

# app.layout = html.Div([html.H1("js",style={'textAlign': 'center'}),
#                        dcc.Dropdown(id='restaurant-dropdown',
#                                     options=[{'label': i, 'value': i} for i in bar.index.unique()],
#                                     value='a'),
#                        html.Div([html.Div(children = [dcc.Graph(id='graph')],
#                                           className='six columns'),
#                                  html.Div(children = [dcc.Graph(id='radar')],
#                                           className='six columns')]),
#                     #    html.Div(DSIH(df[df.name == 'Lost Lemons'].to_html())),
#                        ])
# # Define callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     Output('radar', 'figure'),
#     [Input("restaurant-dropdown", "value")]
# )

# def update_figure(drop):
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=df[df.name == drop].index,
#                              y=df[df.name == drop].sentiment,
#                              mode='lines',
#                              name='confirmed',line=dict( width=1)))
#     fig.update_layout(margin=dict(l=0,r=20,b=0,t=100,pad=0),
#                       paper_bgcolor="white",height= 500,
#                       legend=dict(x=.01,y=.98),
#                       title_text = 'reviews',font_size=15,
#                       xaxis_title="name",
#                       yaxis_title="Number of reviews")
#     da = pd.DataFrame({'r':df[df.name == drop].sentiment,'theta':df[df.name == drop].index})
#     radar = go.Figure()
#     radar.add_trace(go.Scatterpolar(r=da['r'],theta=da['theta'],mode='lines',fill='toself'))
#     radar.update_layout(height=500)
#     return [fig,radar]

# app.run_server(debug=True)#,mode = "inline")

# # css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # app = JupyterDash(__name__,external_stylesheets=css)
# # app.layout = html.Div([html.Div([html.Div(className="six columns",
# #                                           children = [html.H3('Column 1'),
# #                                                       dcc.Graph(id='g1', figure={'data': [{'y': [1, 2, 3]}]})]),
# #                                  html.Div([html.H3('Column 2'),
# #                                            dcc.Graph(id='g2', figure={'data': [{'y': [1, 2, 3]}]})],
# #                                            className="six columns"),],
# #                                  className="row")])
# # app.css.append_css({
# #     'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
# # })

# # if __name__ == '__main__':
# #     app.run_server(debug=True)

In [11]:
# da = pd.DataFrame({'r':[1, 2, 3],'theta':['a', 'b', 'c']})
# radar = go.Figure()
# radar.add_trace(go.Scatterpolar(r=da['r'],theta=da['theta'],mode='lines',fill='toself'))
# radar.update_layout(height=500)

In [13]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [14]:
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [41]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
        <!DOCTYPE html>
        <html>
            <head>
                <title>My App</title>
            </head>
            <body>
                <h1>Addition of two numbers</h1>
                <p id="demo2"></p>
                {app_entry}
                {config}
                {scripts}
                <script>var x=5;
                var y=6;
                var z;
                z=x+y;
                document.getElementById("demo2").innerHTML="Sum of 5 and 6 is "+ z;</script>
                {renderer}
                <div id="custom-footer">My custom footer</div>
            </body>
        </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simple Dash App')

app.run_server(debug=True,port = 8060)


Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
        <!DOCTYPE html>
        <html>
            <head>
                <title>My App</title>
            </head>
            <body>
            <div id="textbox2">My custom sasasasas</div>
            <h1>Addition of two numbers</h1>
            <p id="demo2"></p>
            <div id="custom-footer">My custom footer</div>
            </body>
        </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simple Dash App')

app.run_server(debug=True,port = 8060)

In [16]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)